In [3]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
# faster rcnn model이 포함된 library
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm

In [2]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        # torchvision faster_rcnn은 label=0을 background로 취급
        # class_id를 1~10으로 수정 
        labels = np.array([x['category_id']+1 for x in anns]) 
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
                                
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)

        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.tensor(sample['bboxes'], dtype=torch.float32)

        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [3]:
def get_train_transform():
    return A.Compose([
        A.Resize(1024, 1024),
        A.Flip(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [10]:
import json
import pandas as pd
from collections import defaultdict

def make_dataframe(target):
    # target 에 존재하는 train.json 파일을 엽니다.
    with open(target, 'r') as f:
        json_datas = json.load(f) # python dict 처럼 접근하게끔 변환
    		#dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])

    category = {}
    file_info = {}
    make_frame = defaultdict(list)

    # 이미지 정보 중 파일 경로와 아이디만 추출해서 file_info 에 저장
    for item in json_datas['images']:
        file_info[item['id']] = {'id' : item['id'], 'file_name' : item['file_name']}
    # 카테고리 정보를 category 에 저장
    for item in json_datas['categories']:
        category[item['id']] = item['name']
    # annotations 에 속하는 아이템들을 images 에 속하는 아이템의 정보와 합치기 위함
    for annotation in json_datas['annotations']:
        save_dict = file_info[annotation['image_id']]
        # 각 이미지에 해당하는 bounding box 정보와 class 정보 area(넓이) 정보를 추가
        bbox = np.array(annotation['bbox'])
        bbox[2] = bbox[2] + bbox[0]
        bbox[3] = bbox[3] + bbox[1]
        save_dict.update({
            'class': annotation['category_id'], # 배경은 0, 나머지 +1 in faster_rcnn
            'x_min': bbox[0],
            'y_min': bbox[1],
            'x_max': bbox[2],
            'y_max': bbox[3],
            'area':annotation['area']
            })

        for k,v in save_dict.items():
            # dataframe 으로 만들기 위해서 'key' : [item1,item2...] 형태로 저장
            make_frame[k].append(v)

    # dictionary 가 잘 만들어 졌는지 길이를 측정해서 확인해보세요!
    print(len(json_datas['annotations']))
    # dictionary to DataFrame
    df = pd.DataFrame.from_dict(make_frame)
    df.to_csv('./detection_info.csv',index=False)
    print(df.head())

    return df

In [11]:
annotation = '../dataset/train.json' # annotation 경로
data_dir = '../dataset' # data_dir 경로
train_df = make_dataframe(annotation)


23144
   id       file_name  class  x_min  y_min  x_max  y_max       area
0   0  train/0000.jpg      0  197.6  193.7  745.4  663.4  257301.66
1   1  train/0001.jpg      3    0.0  407.4   57.6  588.0   10402.56
2   1  train/0001.jpg      7    0.0  455.6  144.6  637.2   26259.36
3   1  train/0001.jpg      4  722.3  313.4  996.6  565.3   69096.17
4   1  train/0001.jpg      5  353.2  671.0  586.9  774.4   24164.58


In [12]:
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict


In [13]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(train_y) + 1 # class num ()
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num)) # 그룹마다 클래스의 수 분포를 파악
    y_distr = Counter() # 모든 라벨의 개수를 세어서 dict 형태로 반환 

    for label, g in zip(train_y, groups):
        y_counts_per_group[g][label] += 1 # 그룹마다 라벨의 위치에 클래스 개수를 늘려준다.
        y_distr[label] += 1 # 라벨의 개수 증가
    #print(y_counts_per_group["train/4882.jpg"])
    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num)) # fold마다 클래스의 수 분포를 파악
    groups_per_fold = defaultdict(set) # fold별 group 만들기

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())

    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices



In [14]:
train_x = train_df["id"]
train_y = train_df["class"]
groups = train_df["file_name"]

def get_distribution(y_vals):
        y_distr = Counter(y_vals)
        y_vals_sum = sum(y_distr.values())
        return [f'{y_distr[i] / y_vals_sum:.2%}' for i in range(np.max(y_vals) + 1)]


In [15]:

distrs = [get_distribution(train_y)]
index = ['training set']

for fold_ind, (dev_ind, val_ind) in enumerate(stratified_group_k_fold(train_x, train_y, groups, k=5)):
    dev_y, val_y = train_y[dev_ind], train_y[val_ind]
    dev_groups, val_groups = groups[dev_ind], groups[val_ind]
    
    assert len(set(dev_groups) & set(val_groups)) == 0
    
    distrs.append(get_distribution(dev_y))
    index.append(f'development set - fold {fold_ind}')
    distrs.append(get_distribution(val_y))
    index.append(f'validation set - fold {fold_ind}')

    


In [16]:

print('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(np.max(train_y) + 1)])

Distribution per class:


,Label 0,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7,Label 8,Label 9
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
development set - fold 0,17.14%,27.45%,3.87%,4.04%,4.24%,12.72%,5.46%,22.38%,0.69%,2.02%
validation set - fold 0,17.13%,27.43%,3.88%,4.06%,4.25%,12.71%,5.46%,22.36%,0.69%,2.03%
development set - fold 1,17.14%,27.45%,3.87%,4.05%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
validation set - fold 1,17.12%,27.44%,3.89%,4.04%,4.25%,12.72%,5.46%,22.37%,0.69%,2.03%
development set - fold 2,17.14%,27.45%,3.88%,4.05%,4.25%,12.71%,5.46%,22.37%,0.69%,2.02%
validation set - fold 2,17.13%,27.44%,3.87%,4.04%,4.23%,12.72%,5.47%,22.38%,0.69%,2.03%
development set - fold 3,17.13%,27.44%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
validation set - fold 3,17.15%,27.46%,3.87%,4.04%,4.24%,12.71%,5.45%,22.38%,0.69%,2.01%
development set - fold 4,17.13%,27.44%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
